# Formula algorithm 2 (with last interaction)

In [1]:
import math
import pandas as pd
import numpy as np
import csv
import pandasql as ps

In [2]:
train_data = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/train.csv")

In [5]:
train = train_data

In [6]:
train_exercise = train[:100000]
train_exercise.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


`get last interaction`

In [5]:
train_exercise['last_session_click'] = train_exercise.groupby(['user_id','session_id'])['step'].transform(lambda x: x == x.iloc[-1]).astype(int)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
train_exercise['last_item_interacted'] = train_exercise.loc[train_exercise['last_session_click'].shift(-1)==1, 'reference']
train_exercise['last_item_interacted'] = train_exercise['last_item_interacted'].shift()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
train_exercise.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


`explode to compute stats`

In [10]:
train_exercise_expl = explode_item_price(train_exercise, 'impressions', 'prices')

In [11]:
train_exercise_expl = train_exercise_expl.astype(str)

In [12]:
train_exercise_expl['is_clicked'] = np.where(train_exercise_expl['reference'] == train_exercise_expl['impressions'], 1, 0)
train_exercise_expl['last_interacted_flag'] = np.where(train_exercise_expl['last_item_interacted'] == train_exercise_expl['impressions'], 0.5, 1)


In [13]:
train_exercise_expl.groupby(['last_interacted_flag','is_clicked']).count()

user_id  session_id  timestamp    step  \
last_interacted_flag is_clicked                                           
0.5                  0               732         732        732     732   
                     1              1246        1246       1246    1246   
1.0                  0            203424      203424     203424  203424   
                     1              8031        8031       8031    8031   

                                 action_type  reference  platform    city  \
last_interacted_flag is_clicked                                             
0.5                  0                   732        732       732     732   
                     1                  1246       1246      1246    1246   
1.0                  0                203424     203424    203424  203424   
                     1                  8031       8031      8031    8031   

                                 device  current_filters  last_session_click  \
last_interacted_flag is_clicked                                                
0.5                  0              732              732                 732   
                     1             1246             1246                1246   
1.0                  0           203424           203424              203424   
                     1             8031             8031                8031   

                                 last_item_interacted  impressions  prices  
last_interacted_flag is_clicked                                             
0.5                  0                            732          732     732  
                     1                           1246         1246    1246  
1.0                  0                         203424       203424  203424  
                     1                           8031         8031    8031

## creating the formula 2.0

In [24]:
GR_COLS = ["user_id", "session_id", "timestamp", "step"]

In [15]:
train_exercise = train[:100000]
train_exercise['last_session_click'] = train_exercise.groupby(['user_id','session_id'])['step'].transform(lambda x: x == x.iloc[-1]).astype(int)
train_exercise['last_item_interacted'] = train_exercise.loc[train_exercise['last_session_click'].shift(-1)==1, 'reference']
train_exercise['last_item_interacted'] = train_exercise['last_item_interacted'].shift()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [16]:
train_exercise.head(1000)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,last_session_click,last_item_interacted
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,0,NaN


### start of formula

In [9]:
test = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/test_new.csv")

In [10]:
test['last_session_click'] = test.groupby(['user_id','session_id'])['step'].transform(lambda x: x == x.iloc[-1]).astype(int)
test['last_item_interacted'] = test.loc[test['last_session_click'].shift(-1)==1, 'reference']
test['last_item_interacted'] = test['last_item_interacted'].shift()


In [11]:
test_with_last_interact = test

In [14]:
df_target_price = get_submission_target(test)

In [17]:
df_explode_price = explode_item_price(df_target_price, 'impressions','prices')
df_explode_price.impressions = df_explode_price.impressions.astype(str)

In [18]:
df_explode_price['last_interacted_flag'] = np.where(df_explode_price['last_item_interacted'] == df_explode_price['impressions'], 0.3, 1)
df_explode_price.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,last_session_click,last_item_interacted,impressions,prices,last_interacted_flag
0,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,1,3241426,2059240,70,1.0
1,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,1,3241426,2033381,46,1.0
2,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,1,3241426,1724779,48,1.0
3,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,1,3241426,127131,76,1.0
4,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,1,3241426,399441,65,1.0


In [19]:
df_explode_price["POS"] = (
    df_explode_price
    .groupby(['user_id','session_id','timestamp','step'])
    .cumcount()
)

In [20]:
df_explode_price = (
    df_explode_price
    .assign(position = df_explode_price.POS + 1)
)

In [25]:
df_explode_price['price_mean'] = (
    df_explode_price
    .groupby(GR_COLS)['prices']
    .transform('mean')
)

In [26]:
df_explode_price = (
    df_explode_price
    .assign(log_position = np.log(df_explode_price.position))
)

In [27]:
df_explode_price = (
    df_explode_price
    .assign(price_ratio = df_explode_price.prices / df_explode_price.price_mean)
)

In [28]:
df_explode_price = (
    df_explode_price
    .assign(formula_3 = df_explode_price.log_position * df_explode_price.price_ratio * df_explode_price.last_interacted_flag)
)

In [29]:
df_explode_price.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,...,last_item_interacted,impressions,prices,last_interacted_flag,POS,position,log_position,price_mean,price_ratio,formula_3
0,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,...,3241426,2059240,70,1.0,0,1,0.000000,62.6,1.118211,0.000000
1,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,...,3241426,2033381,46,1.0,1,2,0.693147,62.6,0.734824,0.509341
2,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,...,3241426,1724779,48,1.0,2,3,1.098612,62.6,0.766773,0.842386
3,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,...,3241426,127131,76,1.0,3,4,1.386294,62.6,1.214058,1.683041
4,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,...,3241426,399441,65,1.0,4,5,1.609438,62.6,1.038339,1.671142


In [33]:
# takes a lot of time
df_expl_price = (
    df_explode_price
    .groupby(['user_id','session_id','timestamp','step'])
    .apply(lambda x: x.sort_values(['formula_3','POS']))
    .reset_index(drop=True)
)
# takes a lot of time

In [34]:
df_expl_price = df_expl_price[GR_COLS + ["impressions"]]


In [35]:
df_out1 = df_expl_price.assign(impressions=lambda x: x["impressions"].apply(str))


In [38]:
df_out = group_concat(df_out1, GR_COLS, 'impressions')


In [39]:
df_out.rename(columns={'impressions': 'item_recommendations'}, inplace=True)
df_out[:10]

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,923407 1729121 1241375 1050792 97171 2182130 3...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,2213014 3184842 2292254 3812004 8153310 320289...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,3812004 3505150 2292254 3202894 4476010 710135...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22721 22854 16121 22819 3067559 22764 1478189 ...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,507861 2176280 1669587 4272108 3133204 502066 ...
5,000GO9NY6P4M,55dbafdbb9bab,1541594662,2,157710 319866 1287958 160577 483691 1618677 40...
6,000IRHJS2DL9,f6ffffd20d43d,1541605541,12,102540 2285010 33191 20144 2547840 4481844 201...
7,000JB0UNEH23,7df07dc9fe26e,1541618174,1,2184212 2756867 4063406 9503248 3884098 290999...
8,000OWRCYEHKT,53e84da5c2dad,1541706095,3,5416250 7795438 3926654 8989706 6797450 722521...
9,000VBY1D6BP8,033fddaaa99af,1541587306,4,32256 32246 5411980 6865398 32230 5592656 3223...


In [16]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_item_price(df_in, col_explode1, col_explode2):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_explode1] = df[col_explode1].apply(string_to_array)
    df.loc[:, col_explode2] = df[col_explode2].apply(string_to_array)    

    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_explode1].str.len())
         for col in df.columns.drop([col_explode1,col_explode2])}
    )

    df_out.loc[:, col_explode1] = np.concatenate(df[col_explode1].values)
    df_out.loc[:, col_explode2] = np.concatenate(df[col_explode2].values)  
    
    df_out.loc[:, col_explode1] = df_out[col_explode1].apply(int)
    df_out.loc[:, col_explode2] = df_out[col_explode2].apply(int)

    return df_out

In [13]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [37]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out